# Introduction to CNN Keras - Acc 0.997 (top 8%) 
### **Yassine Ghouzam, PhD**
#### 18/07/2017

* **1. Introduction**
* **2. Data preparation**
    * 2.1 Load data
    * 2.2 Check for null and missing values
    * 2.3 Normalization
    * 2.4 Reshape
    * 2.5 Label encoding
    * 2.6 Split training and valdiation set
* **3. CNN**
    * 3.1 Define the model
    * 3.2 Set the optimizer and annealer
    * 3.3 Data augmentation
* **4. Evaluate the model**
    * 4.1 Training and validation curves
    * 4.2 Confusion matrix
* **5. Prediction and submition**
    * 5.1 Predict and Submit results

# 1. Introducción

Esta es una red neuronal convolucional secuencial de 5 capas para el reconocimiento de dígitos entrenada en el conjunto de datos MNIST. Elegí construirlo con la API de keras (backend de Tensorflow), que es muy intuitivo. En primer lugar, prepararé los datos (imágenes de dígitos escritos a mano) y luego me centraré en el modelado y la evaluación de CNN.

Logré un 99.671% de precisión con esta CNN entrenada en 2h30 en una sola CPU (i5 2500k). Para aquellos que tienen capacidades de GPU >= 3.0 (desde GTX 650 hasta GPU recientes), pueden usar tensorflow-gpu con keras. ¡El cálculo será mucho más rápido!

**Por razones de cálculo, configuré el número de pasos (épocas) en 2, si desea lograr más del 99 % de precisión, configúrelo en 30.**

Este Cuaderno consta de tres partes principales:

* La preparación de datos
* El modelado y evaluación de CNN
* La predicción y presentación de resultados.




<img src="http://img1.imagilive.com/0717/mnist-sample.png" ></img>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau


sns.set(style='white', context='notebook', palette='deep')

# 2. Preparación de datos
## 2.1 Cargar datos

In [2]:
# Cargar los datos
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

In [3]:
Y_train = train["label"]

# eliminar la columna 'etiqueta'
X_train = train.drop(labels = ["label"],axis = 1) 

# libera algo de espacio
del train 

g = sns.countplot(Y_train)

Y_train.value_counts()

Tenemos conteos similares para los 10 dígitos.

## 2.2 Comprobar valores nulos y faltantes

In [4]:
# Revisa los datos
X_train.isnull().any().describe()

In [5]:
test.isnull().any().describe()

Compruebo si hay imágenes dañadas (faltan valores en el interior).

No faltan valores en el conjunto de datos de tren y prueba. Para que podamos seguir adelante con seguridad.

## 2.3 Normalización
Realizamos una normalización en escala de grises para reducir el efecto de las diferencias de iluminación.

Además, la CNN converge más rápido en [0..1] datos que en [0..255].

In [6]:
# Normalizar los datos
X_train = X_train / 255.0
test = test / 255.0

## 2.3 Reshape

In [7]:
# Reformar la imagen a 3 dimensiones (height = 28px, width = 28px , canal = 1)
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

Las imágenes de entrenamiento y prueba (28 px x 28 px) se han almacenado en pandas. Dataframe como vectores 1D de 784 valores. Reformamos todos los datos a matrices 3D de 28x28x1.

Keras requiere una dimensión extra al final que corresponde a los canales. Las imágenes MNIST están en escala de grises, por lo que solo usan un canal. Para imágenes RGB, hay 3 canales, habríamos remodelado vectores de 784 px a matrices 3D de 28x28x3.

## 2.5 Label encoding

In [8]:
# Codificar etiquetas en un vector activo (por ejemplo: 2 -> [0,0,1,0,0,0,0,0,0,0])
Y_train = to_categorical(Y_train, num_classes = 10)

Las etiquetas son números de 10 dígitos del 0 al 9. Necesitamos codificar las etiquetas en un vector activo (por ejemplo: 2 -> [0,0,1,0,0,0,0,0,0,0]).

## 2.6 Conjunto dividido de entrenamiento y validación

In [9]:
# Establecer la semilla aleatoria
random_seed = 2

In [10]:
# Dividir el tren y el conjunto de validación para el ajuste
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=random_seed)

I choosed to split the train set in two parts : a small fraction (10%) became the validation set which the model is evaluated and the rest (90%) is used to train the model.

Since we have 42 000 training images of balanced labels (see 2.1 Load data), a random split of the train set doesn't cause some labels to be over represented in the validation set. Be carefull with some unbalanced dataset a simple random split could cause inaccurate evaluation during the validation. 

To avoid that, you could use stratify = True option in train_test_split function (**Only for >=0.17 sklearn versions**).

We can get a better sense for one of these examples by visualising the image and looking at the label.

In [11]:
# Algunos ejemplos
g = plt.imshow(X_train[0][:,:,0])

# 3. CNN
## 3.1 Definir el modelo

Usé la API secuencial de Keras, donde solo tiene que agregar una capa a la vez, comenzando desde la entrada.
​
La primera es la capa convolucional (Conv2D). Es como un conjunto de filtros que se pueden aprender. Elegí configurar 32 filtros para las dos primeras capas conv2D y 64 filtros para las dos últimas. Cada filtro transforma una parte de la imagen (definida por el tamaño del núcleo) utilizando el filtro del núcleo. La matriz de filtro kernel se aplica a toda la imagen. Los filtros pueden verse como una transformación de la imagen.
​
La CNN puede aislar características que son útiles en todas partes a partir de estas imágenes transformadas (mapas de características).
​
La segunda capa importante en CNN es la capa de agrupación (MaxPool2D). Esta capa simplemente actúa como un filtro de reducción de resolución. Mira los 2 píxeles vecinos y elige el valor máximo. Estos se utilizan para reducir el costo computacional y, en cierta medida, también reducen el sobreajuste. Tenemos que elegir el tamaño de agrupación (es decir, el tamaño del área agrupada cada vez) cuanto más alta sea la dimensión de agrupación, más importante será la reducción de resolución.
​
Al combinar capas convolucionales y de agrupación, CNN puede combinar características locales y aprender más características globales de la imagen.
​
Dropout es un método de regularización, donde una proporción de nodos en la capa se ignora aleatoriamente (estableciendo sus pesos en cero) para cada muestra de entrenamiento. Esto elimina aleatoriamente una parte de la red y obliga a la red a aprender funciones de forma distribuida. Esta técnica también mejora la generalización y reduce el sobreajuste.
​
'relu' es el rectificador (función de activación max(0,x). La función de activación del rectificador se usa para agregar no linealidad a la red.
​
La capa Flatten se usa para convertir los mapas de características finales en un solo vector 1D. Este paso de aplanamiento es necesario para poder utilizar capas totalmente conectadas después de algunas capas convolucionales/maxpool. Combina todas las características locales encontradas de las capas convolucionales anteriores.
​
Al final, utilicé las funciones en dos capas completamente conectadas (Densas), que es solo un clasificador artificial de redes neuronales (ANN). En la última capa (Dense (10, activación = "softmax")) la red genera la distribución de probabilidad de cada clase.

In [12]:
# Establecer el modelo CNN
# mi arquitectura CNN está en -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out

model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

## 3.2 Configurar el optimizador y recocido

Una vez que nuestras capas se agregan al modelo, debemos configurar una función de puntuación, una función de pérdida y un algoritmo de optimización.

Definimos la función de pérdida para medir qué tan mal funciona nuestro modelo en imágenes con etiquetas conocidas. Es la tasa de error entre las etiquetas observadas y las predichas. Usamos un formulario específico para clasificaciones categóricas (> 2 clases) llamado "categorical_crossentropy".

La función más importante es el optimizador. Esta función mejorará iterativamente los parámetros (filtros de valores del núcleo, pesos y sesgo de las neuronas...) para minimizar la pérdida.

Elegí RMSprop (con valores predeterminados), es un optimizador muy efectivo. La actualización de RMSProp ajusta el método Adagrad de una manera muy simple en un intento de reducir su tasa de aprendizaje agresiva y monótonamente decreciente.
También podríamos haber usado el optimizador Stochastic Gradient Descent ('sgd'), pero es más lento que RMSprop.

La función métrica "precisión" se utiliza para evaluar el rendimiento de nuestro modelo.
Esta función de métrica es similar a la función de pérdida, excepto que los resultados de la evaluación de la métrica no se utilizan al entrenar el modelo (solo para la evaluación).

In [13]:
# Definir el optimizador
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [14]:
# Compilar el modelo
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

Para hacer que el optimizador converja más rápido y más cerca del mínimo global de la función de pérdida, utilicé un método de recocido de la tasa de aprendizaje (LR).

El LR es el paso por el cual el optimizador recorre el 'panorama de pérdidas'. Cuanto mayor sea el LR, mayores serán los pasos y más rápida la convergencia. Sin embargo, el muestreo es muy pobre con un LR alto y el optimizador probablemente podría caer en un mínimo local.

Es mejor tener una tasa de aprendizaje decreciente durante el entrenamiento para alcanzar eficientemente el mínimo global de la función de pérdida.

Para mantener la ventaja del tiempo de cálculo rápido con un LR alto, reduje el LR dinámicamente cada X pasos (épocas) dependiendo si es necesario (cuando no se mejora la precisión).

Con la función ReduceLROnPlateau de Keras.callbacks, elijo reducir el LR a la mitad si la precisión no mejora después de 3 épocas.

In [15]:
# Establecer un recocido de tasa de aprendizaje
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [16]:
epochs = 1 # Gire las épocas a 30 para obtener una precisión de 0.9967
batch_size = 86

## 3.3 Aumento de datos
Para evitar el problema de sobreajuste, necesitamos expandir artificialmente nuestro conjunto de datos de dígitos escritos a mano. Podemos hacer que su conjunto de datos existente sea aún más grande. La idea es alterar los datos de entrenamiento con pequeñas transformaciones para reproducir las variaciones que ocurren cuando alguien está escribiendo un dígito.

Por ejemplo, el número no está centrado.
La escala no es la misma (algunos que escriben con números grandes/pequeños)
La imagen está girada...

Los enfoques que alteran los datos de entrenamiento de manera que cambian la representación de la matriz mientras mantienen la etiqueta igual se conocen como técnicas de aumento de datos. Algunos aumentos populares que la gente usa son las escalas de grises, los giros horizontales, los giros verticales, los recortes aleatorios, los cambios de color, las traslaciones, las rotaciones y mucho más.

Al aplicar solo un par de estas transformaciones a nuestros datos de entrenamiento, podemos duplicar o triplicar fácilmente la cantidad de ejemplos de entrenamiento y crear un modelo muy sólido.

La mejora es importante:
    - Sin aumento de datos obtuve una precisión del 98.114%
    - Con el aumento de datos logré un 99,67 % de precisión

In [17]:
# Sin aumento de datos obtuve una precisión de 0.98114
#history = model.fit(X_train, Y_train, batch_size = batch_size, epochs = epochs,
# validación_datos = (X_val, Y_val), detallado = 2)

In [18]:
# Con aumento de datos para evitar el sobreajuste (precisión 0.99286)

datagen = ImageDataGenerator(
         featurewise_center=False, # establece la media de entrada en 0 sobre el conjunto de datos
         samplewise_center=False, # establecer la media de cada muestra en 0
         featurewise_std_normalization=False, # divide las entradas por el estándar del conjunto de datos
         samplewise_std_normalization=False, # dividir cada entrada por su estándar
         zca_whitening=False, # aplicar blanqueamiento ZCA
         rotación_rango=10, # rotar aleatoriamente las imágenes en el rango (grados, 0 a 180)
         zoom_range = 0.1, # Ampliar imagen aleatoriamente
         width_shift_range=0.1, # cambiar aleatoriamente las imágenes horizontalmente (fracción del ancho total)
         height_shift_range=0.1, # cambia aleatoriamente las imágenes verticalmente (fracción de la altura total)
         horizontal_flip=False, # voltear imágenes aleatoriamente
         vertical_flip=False) # voltear imágenes aleatoriamente


datagen.fit(X_train)

Para el aumento de datos, elegí:
    - Rota aleatoriamente algunas imágenes de entrenamiento 10 grados
    - Zoom aleatorio en un 10% algunas imágenes de entrenamiento
    - Cambia aleatoriamente las imágenes horizontalmente en un 10% del ancho
    - Cambia aleatoriamente las imágenes verticalmente en un 10% de la altura
   
No apliqué un flip_vertical ni un flip_horizontal ya que podría haber llevado a clasificar erróneamente números simétricos como 6 y 9.

Una vez que nuestro modelo está listo, ajustamos el conjunto de datos de entrenamiento.

In [19]:
# Ajustar el modelo
history = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_val,Y_val),
                              verbose = 2, steps_per_epoch=X_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])

# 4. Evaluar el modelo
## 4.1 Curvas de entrenamiento y validación

In [20]:
# Trazar las curvas de pérdida y precisión para entrenamiento y validación
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['acc'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_acc'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

El siguiente código es para trazar curvas de pérdida y precisión para entrenamiento y validación. Desde entonces, configuré epochs = 2 en este portátil.
Te mostraré las curvas de entrenamiento y validación que obtuve del modelo que construí con 30 épocas (2h30)

<img src="http://img1.imagilive.com/0717/mnist_099671_train_val_loss_acc.png"></img>

El modelo alcanza casi el 99% (98,7+%) de precisión en el conjunto de datos de validación después de 2 épocas. La precisión de la validación es mayor que la precisión del entrenamiento casi todo el tiempo durante el entrenamiento. Eso significa que nuestro modelo no se adapta demasiado al conjunto de entrenamiento.

Nuestro modelo está muy bien entrenado !!!
<img src="http://img1.imagilive.com/0717/accuracies1de.jpg"/>

## 4.2 Matriz de confusión
La matriz de confusión puede ser muy útil para ver los inconvenientes de su modelo.

Trazo la matriz de confusión de los resultados de la validación.

In [24]:
# Mira la matriz de confusión

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Predecir los valores del conjunto de datos de validación
Y_pred = modelo.predecir(X_val)
# Convertir clases de predicciones en vectores calientes
Y_pred_classes = np.argmax(Y_pred,eje = 1)
# Convierta las observaciones de validación en un vector caliente
Y_true = np.argmax(Y_val,eje = 1)
# calcular la matriz de confusión
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
# trazar la matriz de confusión
plot_confusion_matrix(confusion_mtx, classes = range(10)) 

Aquí podemos ver que nuestra CNN funciona muy bien en todos los dígitos con pocos errores considerando el tamaño del conjunto de validación (4 200 imágenes).

Sin embargo, parece que nuestra CNN tiene algunos pequeños problemas con los 4 dígitos, están mal clasificados como 9. A veces es muy difícil captar la diferencia entre 4 y 9 cuando las curvas son suaves.

Investiguemos por errores.

Quiero ver los errores más importantes. Para ese propósito, necesito obtener la diferencia entre las probabilidades del valor real y las predichas en los resultados.

In [ ]:
# Mostrar algunos resultados de error

# Los errores son la diferencia entre las etiquetas predichas y las etiquetas verdaderas
errors = (Y_pred_classes - Y_true != 0)

Y_pred_classes_errors = Y_pred_classes[errors]
Y_pred_errors = Y_pred[errors]
Y_true_errors = Y_true[errors]
X_val_errors = X_val[errors]

def display_errors(errors_index,img_errors,pred_errors, obs_errors):
    """ This function shows 6 images with their predicted and real labels"""
    n = 0
    nrows = 2
    ncols = 3
    fig, ax = plt.subplots(nrows,ncols,sharex=True,sharey=True)
    for row in range(nrows):
        for col in range(ncols):
            error = errors_index[n]
            ax[row,col].imshow((img_errors[error]).reshape((28,28)))
            ax[row,col].set_title("Predicted label :{}\nTrue label :{}".format(pred_errors[error],obs_errors[error]))
            n += 1

# Probabilidades de los números predichos incorrectos
Y_pred_errors_prob = np.max(Y_pred_errors, eje = 1)

# Probabilidades pronosticadas de los valores verdaderos en el conjunto de errores
true_prob_errors = np.diagonal(np.take(Y_pred_errors, Y_true_errors, axis=1))

# Diferencia entre la probabilidad de la etiqueta predicha y la etiqueta verdadera
delta_pred_true_errors = Y_pred_errors_prob - true_prob_errors

# Lista ordenada de los errores de prueba delta
sorted_dela_errors = np.argsort(delta_pred_true_errors)

# 6 errores principales
errores_más_importantes = sorted_dela_errors[-6:]

# Mostrar los 6 errores principales
display_errors(most_important_errors, X_val_errors, Y_pred_classes_errors, Y_true_errors)

Los errores más importantes son también los más intrigantes.
​
Para esos seis casos, el modelo no es ridículo. Algunos de estos errores también pueden ser cometidos por humanos, especialmente por uno el 9 que está muy cerca de un 4. El último 9 también es muy engañoso, me parece que es un 0.

In [22]:
# predecir resultados
results = model.predict(test)

# seleccione el índice con la máxima probabilidad
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [23]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("cnn_mnist_datagen.csv",index=False)